In [1]:
import json
import numpy as np
import os
import pandas as pd
import pathlib

In [3]:
BASE_DIR = '/Users/dev-yoshiki/Hobby/paycheck_reader'
CATEGORIES = ['incomes', 'deductions', 'attendances']
CATEGORY = 'incomes'
EMP_NO = 9999999
JSON_DIR_PATH = '/Users/dev-yoshiki/Hobby/paycheck_reader/data/output/json'
JSON_FILES = ['1300_1614120_K_20180425.json', '1300_1614120_K_20180525.json', '1300_1614120_K_20180625.json', '1300_1614120_K_20180725.json', '1300_1614120_K_20180824.json', '1300_1614120_K_20180925.json', '1300_1614120_K_20181025.json', '1300_1614120_K_20181122.json', '1300_1614120_K_20181225.json', '1300_1614120_K_20190125.json', '1300_1614120_K_20190225.json', '1300_1614120_K_20190325.json', '1300_1614120_K_20190425.json', '1300_1614120_K_20190524.json', '1300_1614120_K_20190625.json', '1300_1614120_K_20190725.json', '1300_1614120_K_20190823.json', '1300_1614120_K_20190925.json', '1300_1614120_K_20191025.json', '1300_1614120_K_20191125.json', '1300_1614120_K_20191225.json', '1300_1614120_K_20200124.json', '1300_1614120_K_20200225.json']
OUTPUT_PATH = '/Users/dev-yoshiki/Hobby/paycheck_reader/dev_frontend/json/incomes.json'
PAID_DATE = 'paid_date'

In [22]:
Response = {'incomes': {}, 'deductions': {}, 'attendances': {}}

In [23]:
def updateRes(response, category, table):
    print('response:', response)
    rows = {'rows': list(table.index)}
    columns = {'columns': list(table.columns)}
    # v_array = table.to_numpy()
    v_array = table.values
    values = {'values': v_array.tolist()}

    # print('response:', response)
    # TODO:Output to one file
    response[category].update(rows)
    response[category].update(columns)
    response[category].update(values)
    print(response)
    return response

In [24]:
def create_table(category):
    dataframes = []
    for filename in JSON_FILES:
        dates, keys, values, indexes = [], [], [], []

        file_path = pathlib.Path(JSON_DIR_PATH, filename)
        with open(file_path, 'r') as json_file:
            dict_data = json.load(json_file)

        """Single key extraction"""
        dates, keys, values = [], [], []
        date = dict_data[PAID_DATE]
        for key, value in dict_data[category].items():
            values.append(value)
            keys.append(key)
            dates.append(date)
        df = pd.DataFrame({'date': dates, 'type': keys, 'value': values})
        dataframes.append(df)
    df = pd.concat(dataframes)
    return df

In [25]:
# Create 'incomes' table 
# -----------------------------------------------------------------------
category = CATEGORIES[0]
df = create_table(category)
# pd.pivot_table(df, index='type', columns='date', values='value', fill_value=0)

In [26]:
# Sort data by ranking values statistically.
table = pd.pivot_table(df, index='type', columns='date', values='value', fill_value=0)
table.loc['不足日数時間精算'] = table.loc['不足日数時間精算'].abs()
table.replace(0, np.nan, inplace=True)

## total, mean, std
table['total'] = table.filter(regex='\d{4}-\d{2}-\d{2}').sum(axis=1)
table['total'] = table['total'].astype(int).round(0)
table['mean'] = table.filter(regex='\d{4}-\d{2}-\d{2}').mean(axis=1)
table['mean'] = table['mean'].astype(int).round(0)
table = table.fillna(0)
table['std'] = table.filter(regex='\d{4}-\d{2}-\d{2}').std(axis=1)
table['std'] = (table['std'] / table['mean'] * 100).astype(float).round(2)

# norm
Total = table['total'].sum()
table['norm'] = (table['total'] / Total * 100).round(2)
print('Total:', Total)
print('sum(norm):', table['norm'].sum())

# sort by std
table = table.sort_values(by=['std'], ascending=True)

# add rank params
table['sum_rank'] = table['total'].rank(ascending=False).astype(int).round(0)
table['std_rank'] = table['std'].rank(ascending=True).astype(int).round(0)
table['norm_rank'] = table['norm'].rank(ascending=False).astype(int).round(0)
table['rank_avg'] = table.filter(regex='_rank$').mean(axis=1).astype(float).round(2)
table['Ranking'] = table['rank_avg'].rank(ascending=True).astype(int).round(0)

# sort by std
table = table.sort_values(by=['Ranking'], ascending=True)

print(table[['total', 'mean', 'std', 'norm', 'sum_rank', 'std_rank', 'norm_rank', 'rank_avg', 'Ranking']])

# roll-back negative income
table.loc['不足日数時間精算'] = table.loc['不足日数時間精算'] * -1

# extract sorted table
st = table.filter(regex='\d{4}-\d{2}-\d{2}')
st

Total: 7240934
sum(norm): 100.00999999999999
date             total    mean    std   norm  sum_rank  std_rank  norm_rank  \
type                                                                          
基本給            5100500  221760   1.01  70.44         1         2          1   
Smart&Fun!支援金   230000   10000   0.00   3.18         4         1          4   
新卒住宅補助          500000   25000  34.44   6.91         3         5          3   
時間外勤務手当         936814   44610  55.57  12.94         2        10          2   
通勤手当*           202420   50605  39.24   2.80         5         7          5   
遡及差額             85429   42714  36.73   1.18         6         6          6   
その他支給(課税加算)      50000   50000  20.85   0.69         8         3          8   
休日勤務手当           51585   12896  42.25   0.71         7         8          7   
資格取得(奨励金)        35700   17850  31.72   0.49        10         4         10   
不足日数時間精算         40979   10244  43.17   0.57         9         9          9   
深夜勤務手当 

date,2018-04-25,2018-05-25,2018-06-25,2018-07-25,2018-08-24,2018-09-25,2018-10-25,2018-11-22,2018-12-25,2019-01-25,...,2019-05-24,2019-06-25,2019-07-25,2019-08-23,2019-09-25,2019-10-25,2019-11-25,2019-12-25,2020-01-24,2020-02-25
type,,,,,,,,,,,,,,,,,,,,,
基本給,220000.0,220000.0,220000.0,220000.0,220000.0,220000.0,220000.0,220000.0,220000.0,220000.0,...,220000.0,224500.0,224500.0,224500.0,224500.0,224500.0,224500.0,224500.0,224500.0,224500.0
Smart&Fun!支援金,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,...,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0
新卒住宅補助,0.0,0.0,0.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,...,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0
時間外勤務手当,0.0,1833.0,0.0,20158.0,73300.0,64138.0,64138.0,75133.0,60473.0,23823.0,...,72345.0,32152.0,45390.0,58629.0,49173.0,47282.0,41608.0,49173.0,13239.0,1892.0
通勤手当*,0.0,0.0,0.0,63200.0,0.0,0.0,0.0,0.0,0.0,46120.0,...,0.0,0.0,46120.0,0.0,0.0,0.0,0.0,0.0,46980.0,0.0
遡及差額,0.0,0.0,0.0,75000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10429.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
その他支給(課税加算),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50000.0,0.0,0.0
休日勤務手当,0.0,0.0,0.0,15833.0,15833.0,0.0,15833.0,0.0,0.0,0.0,...,0.0,0.0,4086.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
資格取得(奨励金),0.0,0.0,0.0,25700.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10000.0


In [27]:
# Rename indexes in English
js_row = st.index.values.tolist()
JS_ROW = ['基本給', 'Smart&Fun!支援金', '新卒住宅補助', '時間外勤務手当', '通勤手当*', '遡及差額', 'その他支給(課税加算)', '休日勤務手当', '資格取得(奨励金)', '不足日数時間精算', '深夜勤務手当']
EN_ROW = ['base', 'supp', 'rent', 'over', 'trans', 'past', 'other', 'off', 'cert', 'defi', 'night']

convTable = dict(zip(JS_ROW, EN_ROW))
print(convTable)
en_table = st.rename(index=convTable)
en_table

{'基本給': 'base', 'Smart&Fun!支援金': 'supp', '新卒住宅補助': 'rent', '時間外勤務手当': 'over', '通勤手当*': 'trans', '遡及差額': 'past', 'その他支給(課税加算)': 'other', '休日勤務手当': 'off', '資格取得(奨励金)': 'cert', '不足日数時間精算': 'defi', '深夜勤務手当': 'night'}


date,2018-04-25,2018-05-25,2018-06-25,2018-07-25,2018-08-24,2018-09-25,2018-10-25,2018-11-22,2018-12-25,2019-01-25,...,2019-05-24,2019-06-25,2019-07-25,2019-08-23,2019-09-25,2019-10-25,2019-11-25,2019-12-25,2020-01-24,2020-02-25
type,,,,,,,,,,,,,,,,,,,,,
base,220000.0,220000.0,220000.0,220000.0,220000.0,220000.0,220000.0,220000.0,220000.0,220000.0,...,220000.0,224500.0,224500.0,224500.0,224500.0,224500.0,224500.0,224500.0,224500.0,224500.0
supp,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,...,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0
rent,0.0,0.0,0.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,...,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0
over,0.0,1833.0,0.0,20158.0,73300.0,64138.0,64138.0,75133.0,60473.0,23823.0,...,72345.0,32152.0,45390.0,58629.0,49173.0,47282.0,41608.0,49173.0,13239.0,1892.0
trans,0.0,0.0,0.0,63200.0,0.0,0.0,0.0,0.0,0.0,46120.0,...,0.0,0.0,46120.0,0.0,0.0,0.0,0.0,0.0,46980.0,0.0
past,0.0,0.0,0.0,75000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10429.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
other,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50000.0,0.0,0.0
off,0.0,0.0,0.0,15833.0,15833.0,0.0,15833.0,0.0,0.0,0.0,...,0.0,0.0,4086.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cert,0.0,0.0,0.0,25700.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10000.0


In [28]:
# Tweek values
demoTable = en_table.copy()
en_row = en_table.index.values.tolist() # EN_ROW
WEIGHT = [0.0389, 0.023, 0.0045, 0.0029, 0.0122, 0.0312, 0.012, 0.033, 0.013, 0.00412]

weiTable = dict(zip(EN_ROW, WEIGHT))

for key, val in weiTable.items():
    print(key, val)
    demoTable.loc[key] = (demoTable.loc[key] * val).astype(int).round(0)
demoTable

base 0.0389
supp 0.023
rent 0.0045
over 0.0029
trans 0.0122
past 0.0312
other 0.012
off 0.033
cert 0.013
defi 0.00412


date,2018-04-25,2018-05-25,2018-06-25,2018-07-25,2018-08-24,2018-09-25,2018-10-25,2018-11-22,2018-12-25,2019-01-25,...,2019-05-24,2019-06-25,2019-07-25,2019-08-23,2019-09-25,2019-10-25,2019-11-25,2019-12-25,2020-01-24,2020-02-25
type,,,,,,,,,,,,,,,,,,,,,
base,8558.0,8558.0,8558.0,8558.0,8558.0,8558.0,8558.0,8558.0,8558.0,8558.0,...,8558.0,8733.0,8733.0,8733.0,8733.0,8733.0,8733.0,8733.0,8733.0,8733.0
supp,230.0,230.0,230.0,230.0,230.0,230.0,230.0,230.0,230.0,230.0,...,230.0,230.0,230.0,230.0,230.0,230.0,230.0,230.0,230.0,230.0
rent,0.0,0.0,0.0,112.0,112.0,112.0,112.0,112.0,112.0,112.0,...,112.0,112.0,112.0,112.0,112.0,112.0,112.0,112.0,112.0,112.0
over,0.0,5.0,0.0,58.0,212.0,186.0,186.0,217.0,175.0,69.0,...,209.0,93.0,131.0,170.0,142.0,137.0,120.0,142.0,38.0,5.0
trans,0.0,0.0,0.0,771.0,0.0,0.0,0.0,0.0,0.0,562.0,...,0.0,0.0,562.0,0.0,0.0,0.0,0.0,0.0,573.0,0.0
past,0.0,0.0,0.0,2340.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,325.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
other,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600.0,0.0,0.0
off,0.0,0.0,0.0,522.0,522.0,0.0,522.0,0.0,0.0,0.0,...,0.0,0.0,134.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cert,0.0,0.0,0.0,334.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,130.0


In [29]:
# Update 'incomes' in response
response = updateRes(Response, category, demoTable)
response

response: {'incomes': {}, 'deductions': {}, 'attendances': {}}
{'incomes': {'rows': ['base', 'supp', 'rent', 'over', 'trans', 'past', 'other', 'off', 'cert', 'defi', 'night'], 'columns': ['2018-04-25', '2018-05-25', '2018-06-25', '2018-07-25', '2018-08-24', '2018-09-25', '2018-10-25', '2018-11-22', '2018-12-25', '2019-01-25', '2019-02-25', '2019-03-25', '2019-04-25', '2019-05-24', '2019-06-25', '2019-07-25', '2019-08-23', '2019-09-25', '2019-10-25', '2019-11-25', '2019-12-25', '2020-01-24', '2020-02-25'], 'values': [[8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0], [230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0], [0.0, 0.0, 0.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 1

{'incomes': {'rows': ['base',
   'supp',
   'rent',
   'over',
   'trans',
   'past',
   'other',
   'off',
   'cert',
   'defi',
   'night'],
  'columns': ['2018-04-25',
   '2018-05-25',
   '2018-06-25',
   '2018-07-25',
   '2018-08-24',
   '2018-09-25',
   '2018-10-25',
   '2018-11-22',
   '2018-12-25',
   '2019-01-25',
   '2019-02-25',
   '2019-03-25',
   '2019-04-25',
   '2019-05-24',
   '2019-06-25',
   '2019-07-25',
   '2019-08-23',
   '2019-09-25',
   '2019-10-25',
   '2019-11-25',
   '2019-12-25',
   '2020-01-24',
   '2020-02-25'],
  'values': [[8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0],
   [230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
  

In [30]:
# Create 'deduction' table
# -----------------------------------------------------------------------
cat = CATEGORIES[1]
df = create_table(cat)
df = pd.pivot_table(df, index='type', columns='date', values='value', fill_value=0)
# ddf

In [31]:
# Rename index in english
ja_row = df.index.values.tolist()
ja_row
JA_ROW = ['その他控除(月例･一時)', '住民税', '健康保険料', '厚生年金保険料', '所得税', '社食利用料', '還付徴収額', '雇用保険料']
EN_ROW = ['other_ded', 'resid_tax', 'health_ins', 'pension_ins', 'income_tax', 'cafe_use', 'refund_bil', 'emp_ins' ]

convTable = dict(zip(JA_ROW, EN_ROW))
print(convTable)
sdf = df.rename(index=convTable)
sdf

{'その他控除(月例･一時)': 'other_ded', '住民税': 'resid_tax', '健康保険料': 'health_ins', '厚生年金保険料': 'pension_ins', '所得税': 'income_tax', '社食利用料': 'cafe_use', '還付徴収額': 'refund_bil', '雇用保険料': 'emp_ins'}


date,2018-04-25,2018-05-25,2018-06-25,2018-07-25,2018-08-24,2018-09-25,2018-10-25,2018-11-22,2018-12-25,2019-01-25,...,2019-05-24,2019-06-25,2019-07-25,2019-08-23,2019-09-25,2019-10-25,2019-11-25,2019-12-25,2020-01-24,2020-02-25
type,,,,,,,,,,,,,,,,,,,,,
other_ded,0,0,0,0,0,0,0,0,-1600,0,...,0,0,0,0,0,0,-1600,0,0,0
resid_tax,0,0,0,0,0,0,0,0,0,0,...,0,-14400,-13300,-13300,-13300,-13300,-13300,-13300,-13300,-13300
health_ins,0,-10200,-10200,-10200,-10200,-10200,-10200,-13600,-13600,-13600,...,-13600,-13600,-13600,-13600,-13600,-13600,-13600,-13600,-13600,-13600
pension_ins,0,-21960,-21960,-21960,-21960,-21960,-21960,-29280,-29280,-29280,...,-29280,-29280,-29280,-29280,-29280,-29280,-29280,-29280,-29280,-29280
income_tax,-5810,-4730,-4660,-13140,-8250,-7390,-8340,-7410,-7340,-6010,...,-7770,-6860,-7200,-7450,-7080,-7030,-6810,-9660,-5800,-5760
cafe_use,0,0,-513,-6996,-7174,-7816,-5797,-8137,-8837,-5493,...,-228,0,0,0,0,0,0,0,0,0
refund_bil,0,0,0,0,0,0,0,0,25524,0,...,0,0,0,0,0,0,0,6371,0,0
emp_ins,-690,-695,-690,-1287,-992,-919,-997,-953,-947,-974,...,-984,-907,-1074,-956,-926,-921,-903,-928,-959,-784


In [32]:
# Tweek values
demoTable = sdf.copy()
en_row = demoTable.index.values.tolist() # EN_ROW
WEIGHT = [0.0389, 0.023, 0.0045, 0.0029, 0.0122, 0.0312, 0.012, 0.033]

weiTable = dict(zip(EN_ROW, WEIGHT))

for key, val in weiTable.items():
    print(key, val)
    demoTable.loc[key] = (demoTable.loc[key] * val).astype(int).round(0)
demoTable

other_ded 0.0389
resid_tax 0.023
health_ins 0.0045
pension_ins 0.0029
income_tax 0.0122
cafe_use 0.0312
refund_bil 0.012
emp_ins 0.033


date,2018-04-25,2018-05-25,2018-06-25,2018-07-25,2018-08-24,2018-09-25,2018-10-25,2018-11-22,2018-12-25,2019-01-25,...,2019-05-24,2019-06-25,2019-07-25,2019-08-23,2019-09-25,2019-10-25,2019-11-25,2019-12-25,2020-01-24,2020-02-25
type,,,,,,,,,,,,,,,,,,,,,
other_ded,0,0,0,0,0,0,0,0,-62,0,...,0,0,0,0,0,0,-62,0,0,0
resid_tax,0,0,0,0,0,0,0,0,0,0,...,0,-331,-305,-305,-305,-305,-305,-305,-305,-305
health_ins,0,-45,-45,-45,-45,-45,-45,-61,-61,-61,...,-61,-61,-61,-61,-61,-61,-61,-61,-61,-61
pension_ins,0,-63,-63,-63,-63,-63,-63,-84,-84,-84,...,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84
income_tax,-70,-57,-56,-160,-100,-90,-101,-90,-89,-73,...,-94,-83,-87,-90,-86,-85,-83,-117,-70,-70
cafe_use,0,0,-16,-218,-223,-243,-180,-253,-275,-171,...,-7,0,0,0,0,0,0,0,0,0
refund_bil,0,0,0,0,0,0,0,0,306,0,...,0,0,0,0,0,0,0,76,0,0
emp_ins,-22,-22,-22,-42,-32,-30,-32,-31,-31,-32,...,-32,-29,-35,-31,-30,-30,-29,-30,-31,-25


In [33]:
# Update response deduc 
# print('Response:', Response)
# print('cat:', cat)
# print('ddf:', ddf)
response = updateRes(Response, cat, demoTable)
response

response: {'incomes': {'rows': ['base', 'supp', 'rent', 'over', 'trans', 'past', 'other', 'off', 'cert', 'defi', 'night'], 'columns': ['2018-04-25', '2018-05-25', '2018-06-25', '2018-07-25', '2018-08-24', '2018-09-25', '2018-10-25', '2018-11-22', '2018-12-25', '2019-01-25', '2019-02-25', '2019-03-25', '2019-04-25', '2019-05-24', '2019-06-25', '2019-07-25', '2019-08-23', '2019-09-25', '2019-10-25', '2019-11-25', '2019-12-25', '2020-01-24', '2020-02-25'], 'values': [[8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0], [230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0], [0.0, 0.0, 0.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0], [0.0, 5.0, 0.0, 58.0, 212

{'incomes': {'rows': ['base',
   'supp',
   'rent',
   'over',
   'trans',
   'past',
   'other',
   'off',
   'cert',
   'defi',
   'night'],
  'columns': ['2018-04-25',
   '2018-05-25',
   '2018-06-25',
   '2018-07-25',
   '2018-08-24',
   '2018-09-25',
   '2018-10-25',
   '2018-11-22',
   '2018-12-25',
   '2019-01-25',
   '2019-02-25',
   '2019-03-25',
   '2019-04-25',
   '2019-05-24',
   '2019-06-25',
   '2019-07-25',
   '2019-08-23',
   '2019-09-25',
   '2019-10-25',
   '2019-11-25',
   '2019-12-25',
   '2020-01-24',
   '2020-02-25'],
  'values': [[8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0],
   [230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
  

In [34]:
# Create 'attendance' table
# -----------------------------------------------------------------------
cat = CATEGORIES[2]
df = create_table(cat)
df = pd.pivot_table(df, index='type', columns='date', values='value', fill_value=0)
df

date,2018-05-25,2018-06-25,2018-07-25,2018-08-24,2018-09-25,2018-10-25,2018-11-22,2018-12-25,2019-01-25,2019-02-25,...,2019-05-24,2019-06-25,2019-07-25,2019-08-23,2019-09-25,2019-10-25,2019-11-25,2019-12-25,2020-01-24,2020-02-25
type,,,,,,,,,,,,,,,,,,,,,
代休取得日数,0.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.0,1.0,1.00
休日勤務時間数,0.00,0.00,8.00,7.45,7.45,8.15,8.15,8.15,8.15,8.15,...,8.15,8.15,1.3,1.30,1.30,1.30,1.30,1.3,1.3,1.30
年休取得日数,0.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00,2.00,1.00,...,1.00,1.00,1.0,1.00,4.00,4.00,4.00,2.0,2.0,2.00
所定内勤務時間数,155.00,155.00,162.45,152.48,160.45,138.00,162.45,162.45,131.45,139.30,...,155.00,147.15,155.0,170.30,131.45,147.15,162.45,139.3,155.0,147.15
所定勤務日数,20.00,20.00,21.00,21.00,23.00,18.00,22.00,21.00,19.00,19.00,...,20.00,19.00,20.0,22.00,21.00,19.00,21.00,20.0,20.0,19.00
所定勤務時間数,155.00,155.00,162.45,162.45,178.15,139.30,170.30,162.45,147.15,147.15,...,155.00,147.15,155.0,170.30,162.45,147.15,162.45,155.0,155.0,147.15
早退回数,0.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.0,1.0,1.00
早退時間,0.00,0.00,0.00,2.12,2.00,1.30,1.30,1.30,1.30,1.30,...,1.30,1.30,1.3,1.30,1.30,1.30,1.30,1.3,1.3,1.30
時間外勤務時間数,0.54,0.54,11.15,39.45,34.56,35.02,40.45,33.15,13.20,31.35,...,39.15,17.10,23.4,31.15,25.46,24.35,22.20,26.1,6.4,0.55


In [35]:
# Rename index in english
ja_row = df.index.values.tolist()
ja_row
JA_ROW = ['代休取得日数', '休日勤務時間数', '年休取得日数', '所定内勤務時間数', '所定勤務日数', '所定勤務時間数', '早退回数', '早退時間', '時間外勤務時間数', '深夜勤務時間数']
EN_ROW = ['make-up_ad', 'holi_wd', 'holi_ad', 'within_wh', 'speci_wd', 'speci_wh', 'early_c', 'early_h', 'over_wh', 'night_wh']

convTable = dict(zip(JA_ROW, EN_ROW))
print(convTable)
rdf = df.rename(index=convTable)
rdf

{'代休取得日数': 'make-up_ad', '休日勤務時間数': 'holi_wd', '年休取得日数': 'holi_ad', '所定内勤務時間数': 'within_wh', '所定勤務日数': 'speci_wd', '所定勤務時間数': 'speci_wh', '早退回数': 'early_c', '早退時間': 'early_h', '時間外勤務時間数': 'over_wh', '深夜勤務時間数': 'night_wh'}


date,2018-05-25,2018-06-25,2018-07-25,2018-08-24,2018-09-25,2018-10-25,2018-11-22,2018-12-25,2019-01-25,2019-02-25,...,2019-05-24,2019-06-25,2019-07-25,2019-08-23,2019-09-25,2019-10-25,2019-11-25,2019-12-25,2020-01-24,2020-02-25
type,,,,,,,,,,,,,,,,,,,,,
make-up_ad,0.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.0,1.0,1.00
holi_wd,0.00,0.00,8.00,7.45,7.45,8.15,8.15,8.15,8.15,8.15,...,8.15,8.15,1.3,1.30,1.30,1.30,1.30,1.3,1.3,1.30
holi_ad,0.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00,2.00,1.00,...,1.00,1.00,1.0,1.00,4.00,4.00,4.00,2.0,2.0,2.00
within_wh,155.00,155.00,162.45,152.48,160.45,138.00,162.45,162.45,131.45,139.30,...,155.00,147.15,155.0,170.30,131.45,147.15,162.45,139.3,155.0,147.15
speci_wd,20.00,20.00,21.00,21.00,23.00,18.00,22.00,21.00,19.00,19.00,...,20.00,19.00,20.0,22.00,21.00,19.00,21.00,20.0,20.0,19.00
speci_wh,155.00,155.00,162.45,162.45,178.15,139.30,170.30,162.45,147.15,147.15,...,155.00,147.15,155.0,170.30,162.45,147.15,162.45,155.0,155.0,147.15
early_c,0.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.0,1.0,1.00
early_h,0.00,0.00,0.00,2.12,2.00,1.30,1.30,1.30,1.30,1.30,...,1.30,1.30,1.3,1.30,1.30,1.30,1.30,1.3,1.3,1.30
over_wh,0.54,0.54,11.15,39.45,34.56,35.02,40.45,33.15,13.20,31.35,...,39.15,17.10,23.4,31.15,25.46,24.35,22.20,26.1,6.4,0.55


In [36]:
# Tweek values
demoTable = rdf.copy()
en_row = demoTable.index.values.tolist() # EN_ROW(10)
WEIGHT = [1, 1, 1, 150, 20, 155, 1, 2, 5, 1]

weiTable = dict(zip(EN_ROW, WEIGHT))

for key, val in weiTable.items():
    print(key, val)
    demoTable.loc[key] = val
demoTable

make-up_ad 1
holi_wd 1
holi_ad 1
within_wh 150
speci_wd 20
speci_wh 155
early_c 1
early_h 2
over_wh 5
night_wh 1


date,2018-05-25,2018-06-25,2018-07-25,2018-08-24,2018-09-25,2018-10-25,2018-11-22,2018-12-25,2019-01-25,2019-02-25,...,2019-05-24,2019-06-25,2019-07-25,2019-08-23,2019-09-25,2019-10-25,2019-11-25,2019-12-25,2020-01-24,2020-02-25
type,,,,,,,,,,,,,,,,,,,,,
make-up_ad,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
holi_wd,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
holi_ad,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
within_wh,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,...,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0
speci_wd,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
speci_wh,155.0,155.0,155.0,155.0,155.0,155.0,155.0,155.0,155.0,155.0,...,155.0,155.0,155.0,155.0,155.0,155.0,155.0,155.0,155.0,155.0
early_c,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
early_h,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
over_wh,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


In [37]:
# Update response 'attend' 
# print('Response:', Response)
# print('cat:', cat)
# print('ddf:', ddf)
response = updateRes(Response, cat, demoTable)
response

response: {'incomes': {'rows': ['base', 'supp', 'rent', 'over', 'trans', 'past', 'other', 'off', 'cert', 'defi', 'night'], 'columns': ['2018-04-25', '2018-05-25', '2018-06-25', '2018-07-25', '2018-08-24', '2018-09-25', '2018-10-25', '2018-11-22', '2018-12-25', '2019-01-25', '2019-02-25', '2019-03-25', '2019-04-25', '2019-05-24', '2019-06-25', '2019-07-25', '2019-08-23', '2019-09-25', '2019-10-25', '2019-11-25', '2019-12-25', '2020-01-24', '2020-02-25'], 'values': [[8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0], [230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0], [0.0, 0.0, 0.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0], [0.0, 5.0, 0.0, 58.0, 212

{'incomes': {'rows': ['base',
   'supp',
   'rent',
   'over',
   'trans',
   'past',
   'other',
   'off',
   'cert',
   'defi',
   'night'],
  'columns': ['2018-04-25',
   '2018-05-25',
   '2018-06-25',
   '2018-07-25',
   '2018-08-24',
   '2018-09-25',
   '2018-10-25',
   '2018-11-22',
   '2018-12-25',
   '2019-01-25',
   '2019-02-25',
   '2019-03-25',
   '2019-04-25',
   '2019-05-24',
   '2019-06-25',
   '2019-07-25',
   '2019-08-23',
   '2019-09-25',
   '2019-10-25',
   '2019-11-25',
   '2019-12-25',
   '2020-01-24',
   '2020-02-25'],
  'values': [[8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8558.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0,
    8733.0],
   [230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
    230.0,
  

In [38]:
# Export combined data to json file
# -----------------------------------------------------------
OUTPUT_PATH = '/Users/dev-yoshiki/Hobby/paycheck_reader/dev_frontend/json/demo_data_200316.json'
jsn = json.dumps(response, ensure_ascii=False)
with open(OUTPUT_PATH, 'w') as json_file:
    json_file.write(jsn)
jsn

'{"incomes": {"rows": ["base", "supp", "rent", "over", "trans", "past", "other", "off", "cert", "defi", "night"], "columns": ["2018-04-25", "2018-05-25", "2018-06-25", "2018-07-25", "2018-08-24", "2018-09-25", "2018-10-25", "2018-11-22", "2018-12-25", "2019-01-25", "2019-02-25", "2019-03-25", "2019-04-25", "2019-05-24", "2019-06-25", "2019-07-25", "2019-08-23", "2019-09-25", "2019-10-25", "2019-11-25", "2019-12-25", "2020-01-24", "2020-02-25"], "values": [[8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8558.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0, 8733.0], [230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0, 230.0], [0.0, 0.0, 0.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112.0], [0.0, 5.0, 0.0, 58.0, 212.0, 186.0